# Using this Py script to get RPA Email Data for reporting
### Last updated by yingyy@ on 2021-06-07

In [11]:
# 需要确认是否安装一下package
import pandas as pd
import re
import numpy as np
import os
import win32com.client
import win32com
import os
import sys
from datetime import datetime
from datetime import timedelta

In [12]:
#扫描rpa邮箱 提取发件箱邮件
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
sent_items = outlook.Folders('rpa@amazon.com').Folders("Sent Items").items

In [13]:
#创建df
info=pd.DataFrame()

In [14]:
# filter message by receive times, 注意这里设定的是提取过去120天邮件
received_dt = datetime.now() - timedelta(days=120)
received_dt = received_dt.strftime('%m/%d/%Y %H:%M %p')
sent_items = sent_items.Restrict("[ReceivedTime] >= '" + received_dt + "'")

In [15]:
# convert email data to df, use n for sample, 按照email格式提取结果邮件
#n = 1
for item in sent_items:
    #n+=1
    temp = []
    b = item.body
    b=b.replace('\n','')
    b=b.split('\r')
    #if n == 200:
    #   break
    if len (b) >= 8:
        index=[0,2,3,4,5,6,7,8]
        for x in index:
            if x == 0:
                temp.append(b[x].replace('Hi,',''))
            else:
                temp.append(b[x].replace(b[x].split(':')[0]+':',''))
        temp=pd.DataFrame(temp).T
        info=pd.concat([info,temp])
    else:
        pass

In [16]:
#命名column, 清洗无用数据
info.columns=['Login','Robot_ID','Email_Received_Time','Start_Time','End_Time','Run_Time','Task_Info','Status']
info['Task_ID'] = info['Status']
info['Source'] = info['Task_Info']
info['Target'] = info['Task_Info']
info = info[info['Login'] != 'Hello,']
info = info[info['Login'] != '']
info = info[info['Login'] != 'RPA']

In [17]:
# info[info.Login.str.contains('zhjingya', case = False)]

In [18]:
# get task id from task info and convert run time, 转换每个task 运行时间到小时数字
for index, row in info.iterrows(): 
    if 'Succeeded' in str(row['Task_ID']):
        row['Task_ID'] = row['Task_ID'].strip().split(' ')[1].strip()
    else:
        row['Task_ID'] = np.nan
    if ':' in str(row['Run_Time']):
        try:
            row['Run_Time'] = float (row['Run_Time'].split(':')[0])+float (row['Run_Time'].split(':')[1])/60+float(row['Run_Time'].split(':')[2])/3600
        except:
            row['Run_Time'] = 'Error'

In [19]:
# get source & target data from task info, 根据不同task id从 task info里面提取source/target
for index, row in info.iterrows(): 
    if row['Task_ID'] == '3':
        try:
            row['Source'] = row['Task_Info'].split('_')[1].strip().upper()
            row['Target'] = row['Task_Info'].split('_')[2].strip().upper()
        except:
            print (row['Task_Info'])
    elif row['Task_ID'] == '4':
        row['Source'] = "US"
        row['Target'] = "AU"
    elif  row['Task_ID'] == '2' or row['Task_ID'] == '6' or row['Task_ID'] == '11':
        row['Source'] = row['Task_Info'].split('_')[1].strip()[:2].upper()
        row['Target'] = row['Task_Info'].split('_')[1].strip()[3:].upper()
    elif row['Task_ID'] == '5':
        row['Source'] = np.nan
        row['Target'] = np.nan
    elif row['Task_ID'] == '8':
        row['Source'] = row['Task_Info'].strip()[:2].upper()
        row['Target'] = row['Task_Info'].strip()[3:5].upper()
    elif row['Task_ID'] == '9':
        row['Source'] = np.nan
        row['Target'] = np.nan
    elif row['Task_ID'] == '10':
        row['Source'] = row['Task_Info'].split('_')[1].upper()
        row['Target'] = row['Task_Info'].split('_')[2].upper()
    else: 
        row['Source'] = np.nan
        row['Target'] = np.nan

In [20]:
#clean rows with empty 'Start_Time' and Error 'Run_Time', 根据之前计算结果删掉无用行
info = info[info['Start_Time'] != '']
info = info[info['Run_Time'] != 'Error']

In [21]:
#生成结果到脚本所在位置excel文件
info.drop_duplicates().to_csv("RPA Report Data.csv", index = False)